##### package install

In [12]:
# %pip install langchain.chat_model
# %pip install langchain
# %pip install openai
# %pip install langchain_community
# %pip install pypdf
# %pip install unstructured
# %pip install python-magic-bin
# %pip install "unstructured[docx]"
# %pip install -U langchain-openai langchain-unstructured


# %pip check
# %pip list | findstr "^langchain"

##### 문서 load

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import UnstructuredFileLoader # 다양한 확장자의 파일을 열 수 있다.

loader = TextLoader("./files/chapter_one.txt")
loader.load()

loader2 = PyPDFLoader("./files/chapter_one.pdf")
loader2.load()

loader3 = UnstructuredFileLoader("./files/chapter_one.docx")
loader3.load()

##### 문서 분할
- 문서의 덩어리가 너무 크기 때문에 나눠야 한다.
    - LLM 에게 질문과 연관된 자료를 줄 때 필요한 '파일의 부분들'만 전달할 수 있다.
    - 답변 생성에 필요한 것을 더 찾기 쉽다.
    - 그러면 Prompt 의 길이도 짧아진다. by 딱 필요한 부분만 주기 때문에

##### 이를 위해
- 문서의 모든 문자를 조각낸다.(모든 문자를 작은 조각들로 만든다.)

In [ ]:
# 일반적인 Text Splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter